# Msticpy and Velociraptor offline collection analysis
BTV Project Obsidian, 2023

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

![BTV Msticpy and Velociraptor offline collection analysis title slide](obsidian-btv-msticpy_velociraptor_offline_collection_analysis.png)

## Agenda

* Codifying analysis
* Key Questions
* Obsidian 2023 logs
  * What is in the environment?
  * Applications, services, browsing...
  * Lnk, MFT, Amcache
  * What matters?

## Codifying analysis
* Security analysis should be documented
* Security analysis should be repeatable
* Security analysis should be verifiable
* Security analysis should use template


### Many options
* Text file and variants
* [Velociraptor notebooks](https://docs.velociraptor.app/docs/vql/notebooks/) on VQL

<div align="center" style="font-weight: bold">
Another option: msticpy<br />
Could merge data from different sources.
</div>


## Key questions
* What happens in my environment
* What's normal
* What's abnormal
* How I would know

## Obsidian 2023 logs

### Code initialization

msticpy PR not merged for now - https://github.com/microsoft/msticpy/pull/668

In [ ]:
# %pip install git+https://github.com/microsoft/msticpy.git@ianhelle/velociraptor-provider-2023-05-19

In [1]:
# Check we are running Python 3.6
import sys

MIN_REQ_PYTHON = (3, 6)
if sys.version_info < MIN_REQ_PYTHON:
    print("Check the Kernel->Change Kernel menu and ensure that Python 3.6")
    print("or later is selected as the active kernel.")
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [2]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime, timedelta
import os

In [3]:
# path to config file
# os.environ["MSTICPYCONFIG"] = "/home/ubuntu/msticpyconfig.yaml"
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
from msticpy.context.geoip import GeoLiteLookup, IPStackLookup

print("Imports Complete")

Imports Complete


In [4]:
# pandas
pd.set_option("display.max_colwidth", 500)

In [5]:
# Ensure to use appropriate filters
query_common_args = ''
results_limit = 10

In [8]:
query_common_args = query_common_args.strip()
# datadir = "/path/to/Collection-iot-eng-wkst_magnumtempus_financial-2023-06-19T01_15_07Z/results"
datadir = "/home/user/Lab1/others/msticpy-data/Collection-iot-eng-wkst_magnumtempus_financial-2023-06-19T01_15_07Z/results"

In [9]:
# Configuration
qry_prov = QueryProvider(
    "VelociraptorLogs",
    data_paths=[
        datadir
    ],
)
qry_prov.connect()

In [10]:
len(qry_prov.list_queries())

25

In [11]:
qry_prov.list_queries()

['velociraptor.Windows_Applications_Chrome_Cookies',
 'velociraptor.Windows_Applications_Chrome_Extensions',
 'velociraptor.Windows_Applications_Chrome_History',
 'velociraptor.Windows_Applications_Edge_History',
 'velociraptor.Windows_EventLogs_ScheduledTasks',
 'velociraptor.Windows_Forensics_Lnk',
 'velociraptor.Windows_Forensics_Usn',
 'velociraptor.Windows_KapeFiles_Targets_2FAll_File_Metadata',
 'velociraptor.Windows_KapeFiles_Targets_2FUploads',
 'velociraptor.Windows_Memory_Acquisition',
 'velociraptor.Windows_NTFS_MFT',
 'velociraptor.Windows_Network_ArpCache',
 'velociraptor.Windows_Network_InterfaceAddresses',
 'velociraptor.Windows_Network_ListeningPorts',
 'velociraptor.Windows_Network_Netstat',
 'velociraptor.Windows_Network_NetstatEnriched_2FNetstat',
 'velociraptor.Windows_Registry_EnabledMacro',
 'velociraptor.Windows_Sys_Users',
 'velociraptor.Windows_Sysinternals_Autoruns',
 'velociraptor.Windows_System_Amcache_2FFile',
 'velociraptor.Windows_System_Amcache_2FInvento

### What is in the environment?
* Linux
* Windows
* Others

### What applications?
* Linux ProcessName
* Windows Event Channel or Source
* Services
* Scheduled tasks
* Browser activities

#### Process

In [12]:
df_pslist = qry_prov.velociraptor.Windows_System_Pslist()

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
df_pslist.head()

,Pid,Ppid,TokenIsElevated,Name,CommandLine,Exe,TokenInfo,Hash,Authenticode,Username,WorkingSetSize
0,4,0,False,System,,,None,"{'MD5': 'd41d8cd98f00b204e9800998ecf8427e', 'SHA1': 'da39a3ee5e6b4b0d3255bfef95601890afd80709', 'SHA256': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'}",None,,147456.0
1,96,4,False,Registry,,,None,"{'MD5': 'd41d8cd98f00b204e9800998ecf8427e', 'SHA1': 'da39a3ee5e6b4b0d3255bfef95601890afd80709', 'SHA256': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'}",None,NT AUTHORITY\SYSTEM,54575104.0
2,308,4,True,smss.exe,\SystemRoot\System32\smss.exe,C:\Windows\System32\smss.exe,None,"{'MD5': '44962fd12f0d29b0713bb5e14653194a', 'SHA1': '3b2af5ef58ecf86abe51a730493363448b95dd0a', 'SHA256': 'd94c6656d377bb43a5cca307fc46f5de2b259e8884acedddf0b07f45425f457e'}","{'Filename': 'C:\Windows\System32\smss.exe', 'ProgramName': 'Microsoft Windows', 'PublisherLink': None, 'MoreInfoLink': 'http://www.microsoft.com/windows', 'SerialNumber': '33000002a5e1a081b7c895c0ed0000000002a5', 'IssuerName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Production PCA 2011', 'SubjectName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Publisher', 'Timestamp': None, 'Trusted': 'trusted', '_ExtraInfo': None}",NT AUTHORITY\SYSTEM,1290240.0
3,432,424,True,csrss.exe,"%SystemRoot%\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,20480,768 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=sxssrv,4 ProfileControl=Off MaxRequestThreads=16",C:\Windows\System32\csrss.exe,None,"{'MD5': 'a6c9ee45bff7c5e696b07ec41af84541', 'SHA1': '7821fb14835083bae87a66ce96b67b261bea6b2b', 'SHA256': '50cbdbce85014f1042f99d5acdaad88ff268317a605bc3700b2487edd0f2ef0e'}","{'Filename': 'C:\Windows\System32\csrss.exe', 'ProgramName': 'Microsoft Windows', 'PublisherLink': None, 'MoreInfoLink': 'http://www.microsoft.com/windows', 'SerialNumber': '33000002a5e1a081b7c895c0ed0000000002a5', 'IssuerName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Production PCA 2011', 'SubjectName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Publisher', 'Timestamp': None, 'Trusted': 'trusted', '_ExtraInfo': None}",NT AUTHORITY\SYSTEM,6639616.0
4,504,424,True,wininit.exe,wininit.exe,C:\Windows\System32\wininit.exe,None,"{'MD5': '6ec6810c6fdae02e87d8a43db98e194d', 'SHA1': '999cdef567bb7497e6206587cf57b95ad8037c75', 'SHA256': '71dd6bfc68e6a840bc935ac08dc71618043bf705849b619d31a2b83e54670a3e'}","{'Filename': 'C:\Windows\System32\wininit.exe', 'ProgramName': 'Microsoft Windows', 'PublisherLink': None, 'MoreInfoLink': 'http://www.microsoft.com/windows', 'SerialNumber': '33000003f6f4e8b30a2be45bd00000000003f6', 'IssuerName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Production PCA 2011', 'SubjectName': 'C=US, ST=Washington, L=Redmond, O=Microsoft Corporation, CN=Microsoft Windows Publisher', 'Timestamp': None, 'Trusted': 'trusted', '_ExtraInfo': None}",NT AUTHORITY\SYSTEM,7376896.0


#### Services

In [14]:
df_services = qry_prov.velociraptor.Windows_System_Services()

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
df_services.head()

,State,Name,DisplayName,Status,Pid,ExitCode,StartMode,PathName,ServiceType,UserAccount,Created,ServiceDll,FailureCommand,FailureActions,AbsoluteExePath,HashServiceExe,CertinfoServiceExe,HashServiceDll,CertinfoServiceDll
0,Stopped,AJRouter,AllJoyn Router Service,OK,0,1077,Manual,C:\Windows\system32\svchost.exe -k LocalServiceNetworkRestricted -p,Share Process,NT AUTHORITY\LocalService,2021-08-19T13:32:39.0871257Z,C:\Windows\System32\AJRouter.dll,None,"{'ResetPeriod': 86400, 'FailureAction': [{'Type': 'SC_ACTION_RESTART', 'Delay': 3}, {'Type': 'SC_ACTION_RESTART', 'Delay': 3}, {'Type': 'SC_ACTION_NONE', 'Delay': 0}]}",C:\Windows\system32\svchost.exe,NaN,NaN,NaN,NaN
1,Stopped,ALG,Application Layer Gateway Service,OK,0,1077,Manual,C:\Windows\System32\alg.exe,Own Process,NT AUTHORITY\LocalService,2021-08-19T13:32:39.0871257Z,None,None,"{'ResetPeriod': 900, 'FailureAction': [{'Type': 'SC_ACTION_RESTART', 'Delay': 120}, {'Type': 'SC_ACTION_RESTART', 'Delay': 300}, {'Type': 'SC_ACTION_NONE', 'Delay': 0}]}",C:\Windows\System32\alg.exe,NaN,NaN,NaN,NaN
2,Stopped,Amazon EC2Launch,Amazon EC2Launch,OK,0,0,Auto,"""C:\Program Files\Amazon\EC2Launch\service\EC2LaunchService.exe""",Own Process,LocalSystem,2023-01-11T04:10:57.3268843Z,None,None,None,C:\Program Files\Amazon\EC2Launch\service\EC2LaunchService.exe,NaN,NaN,NaN,NaN
3,Running,AmazonSSMAgent,Amazon SSM Agent,OK,2472,0,Auto,"""C:\Program Files\Amazon\SSM\amazon-ssm-agent.exe""",Own Process,LocalSystem,2023-04-24T17:18:25.5172819Z,None,None,"{'ResetPeriod': 86400, 'FailureAction': [{'Type': 'SC_ACTION_RESTART', 'Delay': 30}, {'Type': 'SC_ACTION_RESTART', 'Delay': 30}, {'Type': 'SC_ACTION_RESTART', 'Delay': 30}]}",C:\Program Files\Amazon\SSM\amazon-ssm-agent.exe,NaN,NaN,NaN,NaN
4,Stopped,AppIDSvc,Application Identity,OK,0,1077,Manual,C:\Windows\system32\svchost.exe -k LocalServiceNetworkRestricted -p,Share Process,NT Authority\LocalService,2021-08-19T13:32:39.0871257Z,C:\Windows\System32\appidsvc.dll,None,"{'ResetPeriod': 86400, 'FailureAction': [{'Type': 'SC_ACTION_RESTART', 'Delay': 120}, {'Type': 'SC_ACTION_RESTART', 'Delay': 300}, {'Type': 'SC_ACTION_NONE', 'Delay': 0}]}",C:\Windows\system32\svchost.exe,NaN,NaN,NaN,NaN


#### Browser activities

In [16]:
qry_prov.velociraptor.Windows_Applications_Chrome_Extensions()

  0%|          | 0/1 [00:00<?, ?it/s]

,Uid,User,Name,Description,Identifier,Version,Author,Persistent,Path,Scopes,Permissions,Key
0,1170,seth.morgan,Google Docs Offline,"Edit, create, and view your documents, spreadsheets, and presentations — all without internet access.",ghbmnnjooekpmoecnnnilnnbdlolhkhi,1.61.4_0,{'email': 'docs-hosted-app-own@google.com'},0.0,C:\Users\seth.morgan\AppData\Local\Google\Chrome\User Data\Default\Extensions\ghbmnnjooekpmoecnnnilnnbdlolhkhi\1.61.4_0\,None,"[alarms, storage, unlimitedStorage, https://docs.google.com/*, https://drive.google.com/*]",MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAnF7RGLAxIon0/XeNZ4MLdP3DMkoORzEAKVg0sb89JpA/W2osTHr91Wqwdc9lW0mFcSpCYS9Y3e7cUMFo/M2ETASIuZncMiUzX2/0rrWtGQ3UuEj3KSe5PdaVZfisyJw/FebvHwirEWrhqcgzVUj9fL9YjE0G45d1zMKcc1umKvLqPyTznNuKBZ9GJREdGLRJCBmUgCkI8iwtwC+QZTUppmaD50/ksnEUXv+QkgGN07/KoNA5oAgo49Jf1XBoMv4QXtVZQlBYZl84zAsI82hb63a6Gu29U/4qMWDdI7+3Ne5TRvo6Zi3EI4M2NQNplJhik105qrz+eTLJJxvf4slrWwIDAQAB
1,1170,seth.morgan,None,None,nmmhkkegccagdldgiimedpiccmgmieda,1.0.0.6_0,None,NaN,C:\Users\seth.morgan\AppData\Local\Google\Chrome\User Data\Default\Extensions\nmmhkkegccagdldgiimedpiccmgmieda\1.0.0.6_0\,"[https://www.googleapis.com/auth/sierra, https://www.googleapis.com/auth/sierrasandbox, https://www.googleapis.com/auth/chromewebstore, https://www.googleapis.com/auth/chromewebstore.readonly]","[identity, webview, https://www.google.com/, https://www.googleapis.com/*, https://payments.google.com/payments/v4/js/integrator.js, https://sandbox.google.com/payments/v4/js/integrator.js]",MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQCrKfMnLqViEyokd1wk57FxJtW2XXpGXzIHBzv9vQI/01UsuP0IV5/lj0wx7zJ/xcibUgDeIxobvv9XD+zO1MdjMWuqJFcKuSS4Suqkje6u+pMrTSGOSHq1bmBVh0kpToN8YoJs/P/yrRd7FEtAXTaFTGxQL4C385MeXSjaQfiRiQIDAQAB


In [17]:
df_chrome_history = qry_prov.velociraptor.Windows_Applications_Chrome_History()

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
df_chrome_history.head()

,User,url_id,visit_time,visited_url,title,visit_count,typed_count,last_visit_time,hidden,from_url_id,visit_duration,transition,_SourceLastModificationTimestamp,OSPath
0,seth.morgan,1,2023-04-29 21:46:02+00:00,file://172.16.50.146/Users/seth.morgan/Downloads/NEWSHARE2023/2022-Backup/2022-Backup/Internal/Depts/Operations/are-you-ready-guide.pdf,,1,0,2023-04-29 21:46:02+00:00,0,0,00:00:36.330,805306374,2023-04-29T21:46:02Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History
1,seth.morgan,2,2023-04-29 21:46:16+00:00,https://go.microsoft.com/fwlink/?linkid=2132465&form=MT004A&OCID=MT004A,Welcome to Microsoft Edge,2,1,2023-04-29 21:46:16+00:00,0,0,00:00:00.000,268435457,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History
2,seth.morgan,3,2023-04-29 21:46:16+00:00,https://microsoftedgewelcome.microsoft.com/,Welcome to Microsoft Edge,2,0,2023-04-29 21:46:16+00:00,0,2,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History
3,seth.morgan,4,2023-04-29 21:46:16+00:00,https://microsoftedgewelcome.microsoft.com/en-us/,Welcome to Microsoft Edge,2,0,2023-04-29 21:46:16+00:00,0,3,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History
4,seth.morgan,2,2023-04-29 21:46:16+00:00,https://go.microsoft.com/fwlink/?linkid=2132465&form=MT004A&OCID=MT004A,Welcome to Microsoft Edge,2,1,2023-04-29 21:46:16+00:00,0,4,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History


In [19]:
qry_prov.velociraptor.Windows_Applications_Edge_History().head()

  0%|          | 0/1 [00:00<?, ?it/s]

,User,url_id,visit_time,visited_url,title,visit_count,typed_count,last_visit_time,hidden,from_url_id,visit_duration,transition,_SourceLastModificationTimestamp,OSPath,_Source
0,seth.morgan,1,2023-04-29 21:46:02+00:00,file://172.16.50.146/Users/seth.morgan/Downloads/NEWSHARE2023/2022-Backup/2022-Backup/Internal/Depts/Operations/are-you-ready-guide.pdf,,1,0,2023-04-29 21:46:02+00:00,0,0,00:00:36.330,805306374,2023-04-29T21:46:02Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History,Windows.Applications.Chrome.History
1,seth.morgan,2,2023-04-29 21:46:16+00:00,https://go.microsoft.com/fwlink/?linkid=2132465&form=MT004A&OCID=MT004A,Welcome to Microsoft Edge,2,1,2023-04-29 21:46:16+00:00,0,0,00:00:00.000,268435457,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History,Windows.Applications.Chrome.History
2,seth.morgan,3,2023-04-29 21:46:16+00:00,https://microsoftedgewelcome.microsoft.com/,Welcome to Microsoft Edge,2,0,2023-04-29 21:46:16+00:00,0,2,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History,Windows.Applications.Chrome.History
3,seth.morgan,4,2023-04-29 21:46:16+00:00,https://microsoftedgewelcome.microsoft.com/en-us/,Welcome to Microsoft Edge,2,0,2023-04-29 21:46:16+00:00,0,3,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History,Windows.Applications.Chrome.History
4,seth.morgan,2,2023-04-29 21:46:16+00:00,https://go.microsoft.com/fwlink/?linkid=2132465&form=MT004A&OCID=MT004A,Welcome to Microsoft Edge,2,1,2023-04-29 21:46:16+00:00,0,4,00:00:00.000,-2147483647,2023-04-29T21:46:16Z,C:\Users\seth.morgan\AppData\Local\Microsoft\Edge\User Data\Default\History,Windows.Applications.Chrome.History


### Forensics

In [20]:
# Only 3 users: local Administrator, Domain MAGNUMTEMPUS Administrator, seth.morgan
qry_prov.velociraptor.Windows_Sys_Users()

  0%|          | 0/1 [00:00<?, ?it/s]

,Uid,Gid,Name,Description,Directory,UUID,Mtime,HomedirMtime,Data
0,18,,SYSTEM,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-18,%systemroot%\system32\config\systemprofile,S-1-5-18,2021-05-08T08:24:18.988647Z,2021-05-08T08:20:24.3752514Z,"{'ProfileLoadTime': None, 'ProfileUnloadTime': None}"
1,19,,LOCAL SERVICE,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-19,%systemroot%\ServiceProfiles\LocalService,S-1-5-19,2021-05-08T08:24:18.988647Z,2021-08-19T13:31:46.3058657Z,"{'ProfileLoadTime': None, 'ProfileUnloadTime': None}"
2,20,,NETWORK SERVICE,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-20,%systemroot%\ServiceProfiles\NetworkService,S-1-5-20,2021-05-08T08:24:18.988647Z,2021-08-19T13:31:43.2277311Z,"{'ProfileLoadTime': None, 'ProfileUnloadTime': None}"
3,1170,,seth.morgan,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-21-2369732838-3797832421-459094119-1170,C:\Users\seth.morgan,S-1-5-21-2369732838-3797832421-459094119-1170,2023-04-29T22:34:43.4527231Z,2023-04-29T21:29:21.2334698Z,"{'ProfileLoadTime': '2023-04-29T21:35:27Z', 'ProfileUnloadTime': '2023-04-29T22:34:43Z'}"
4,500,,Administrator,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-21-2369732838-3797832421-459094119-500,C:\Users\Administrator.MAGNUMTEMPUS,S-1-5-21-2369732838-3797832421-459094119-500,2023-06-19T01:09:35.6601435Z,2023-06-19T01:10:08.2302754Z,"{'ProfileLoadTime': '2023-06-19T01:09:35Z', 'ProfileUnloadTime': None}"
5,500,,Administrator,HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows NT\CurrentVersion\ProfileList\S-1-5-21-709932091-4166497167-3385689324-500,C:\Users\Administrator,S-1-5-21-709932091-4166497167-3385689324-500,2023-04-29T22:34:43.3389811Z,2023-04-29T18:47:32.908044Z,"{'ProfileLoadTime': '2023-04-29T18:47:17Z', 'ProfileUnloadTime': '2023-04-29T22:34:43Z'}"


In [21]:
df_lnk = qry_prov.velociraptor.Windows_Forensics_Lnk()

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
df_lnk[["WorkingDir", "SourceCreated"]].groupby(["WorkingDir"]).count()

,SourceCreated
WorkingDir,
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\2022-Backup\2022-Backup,1
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\2022-Backup\2022-Backup\Internal\Depts\Operations,1
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\2022-Backup\2022-Backup\SethFolder,1
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\Jason's Private Folder,1
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\Tombstone-Copy\Software,1
\\172.16.50.146\Users\seth.morgan\Downloads\NEWSHARE2023\Tombstone-Copy\Tombstone-Copy,2


In [23]:
df_mft = qry_prov.velociraptor.Windows_NTFS_MFT()

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
df_mft[df_mft["FileName"].str.contains("nmap.exe")]

,EntryNumber,InUse,ParentEntryNumber,OSPath,_Links,FileName,FileSize,ReferenceCount,IsDir,Created0x10,...,LastRecordChange0x10,LastRecordChange0x30,LastAccess0x10,LastAccess0x30,HasADS,SI_Lt_FN,uSecZeros,Copied,FileNames,FileNameTypes
121689,124074,True,123977,\\.\C:\Program Files (x86)\Nmap\nmap.exe,[\\.\C:\Program Files (x86)\Nmap\nmap.exe],nmap.exe,2615440,1,False,2022-09-01T22:36:02Z,...,2023-04-29T21:39:34.0916135Z,2023-04-29T21:39:33.97992Z,2023-04-29T21:39:34.0916135Z,2023-04-29T21:39:33.97992Z,False,True,True,False,[nmap.exe],DOS+Win32
122515,124900,True,123977,\\.\C:\Program Files (x86)\Nmap\zenmap.exe,[\\.\C:\Program Files (x86)\Nmap\zenmap.exe],zenmap.exe,451736,1,False,2022-09-01T22:36:06Z,...,2023-04-29T21:39:41.317326Z,2023-04-29T21:39:41.317326Z,2023-04-29T21:39:41.317326Z,2023-04-29T21:39:41.317326Z,False,True,True,False,[zenmap.exe],DOS+Win32


In [25]:
df_amcache_inv = (
    qry_prov.velociraptor.Windows_System_Amcache_2FInventoryApplicationFile()
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
df_amcache_inv[df_amcache_inv["Binary"].str.contains("nmap")]

,FileId,Key,Hive,LastModified,Binary,Name,Size,ProductName,Publisher,Version,BinFileVersion
69,,\Root\InventoryApplicationFile\ncat.exe|ee87626eac0e966d,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\ncat.exe,ncat.exe,327312,None,None,None,None
70,,\Root\InventoryApplicationFile\ndiff.exe|69ed2f1682772a38,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\ndiff.exe,ndiff.exe,31376,zenmap,None,7.93,7.93.0.0
72,0000ab2de49f90330cc3b305457a9a0f897f296e95f4,\Root\InventoryApplicationFile\nmap-7.93-setup.|f8918f85502c0c40,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\users\seth.morgan\downloads\nmap-7.93-setup.exe,nmap-7.93-setup.exe,29115768,nmap,insecure.org,7.93,7.93.0.0
73,,\Root\InventoryApplicationFile\nmap.exe|ca2723a4280e6400,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\nmap.exe,nmap.exe,2615440,nmap,insecure.org,7.93,7.93.0.0
75,,\Root\InventoryApplicationFile\nping.exe|aea212f892503f7e,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\nping.exe,nping.exe,349328,None,None,None,None
124,,\Root\InventoryApplicationFile\uninstall.exe|a4d3e1dc723a18d,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\uninstall.exe,Uninstall.exe,93592,nmap,insecure.org,7.93,7.93.0.0
160,,\Root\InventoryApplicationFile\zenmap.exe|17e1b52ee953118e,C:\Windows\appcompat\Programs\Amcache.hve,2023-06-19T01:11:06Z,c:\program files (x86)\nmap\zenmap.exe,zenmap.exe,451736,zenmap,None,7.93,7.93.0.0


### What matters?
* Findings
  * From amcache and MFT, we can confirm the install of nmap.
  * From Windows_Forensics_Lnk, we have the list of documents accessed by user: org email list.txt, passwords.xls, are-you-ready-guide.pdf, copy-net-picture.png, admin.txt, Winger_11.jpg, Rig_Notes.xlsx
  * Limited browsing history but matching are-you-ready-guide.pdf
* Combine with other data sources
  * Match with your central logs
* Multiple findings solidify evidence proof

### Share and Report!
* Written documentation
* Structure initial analysis
* Identified anomalies and gaps

## That's all folks!
Thanks to
* [BTV project obsidian](https://www.blueteamvillage.org/programs/project-obsidian/) contributors
* Projects jupyter notebook, pandas, msticpy, OTRF / [Infosec Jupyterthon](https://infosecjupyterthon.com)

<h1 align="center" style="font-weight: bold;">Thank you</h1>

<div align="center">
Join the conversation<br />
https://discord.blueteamvillage.org<br />
Please Provide Feedback<br />
feedback-obsidian@blueteamvillage.org<br />
</div>

## References

* [Velociraptor data provider, msticpy PR](https://github.com/microsoft/msticpy/pull/668)